<a href="https://colab.research.google.com/github/dangngocchau/sentiment-analysis/blob/main/SentimentClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('./content')

Mounted at ./content


In [13]:
import tensorflow_datasets as tfds
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [14]:
imdb, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteTSNJLZ/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteTSNJLZ/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [15]:
import numpy as np
# Lấy ra bộ data train và test từ thư viện dữ liệu imdb reviews
train_data, test_data = imdb['train'], imdb['test']

In [16]:
sample_data = []

for s, l in train_data:
  sample_data.append(str(s.numpy()))

In [ ]:
sample_data[0]

'b"This was an absolutely terrible movie. Don\'t be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie\'s ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor\'s like Christopher Walken\'s good name. I could barely sit through it."'

In [17]:
from bs4 import BeautifulSoup
import re
# Xóa thẻ html
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

# Xóa ngoặc vuông
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
# Xóa kí tự đặc biệt
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text

def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_special_characters(text)
    return text


In [18]:
# Xây dựng từ điển
train_sentences = []
test_sentences = []

train_labels = []
test_labels = []

for s, l in train_data:
  train_sentences.append(str(denoise_text(s.numpy())))
  train_labels.append(l.numpy())
for s, l in test_data:
   test_sentences.append(str(denoise_text(s.numpy())))
   test_labels.append(l.numpy())

In [22]:
def remove_stopword(sentences):
    new_sentences = []
    for sentence in sentences:
      temp = ''
      for word in sentence.split():
          if word not in stopwords.words('english'):
              temp+=word+' '
      new_sentences.append(temp)
    return new_sentences
#train_sentences_new = remove_stopword(train_sentences)

In [24]:
#test_sentences_new = remove_stopword(test_sentences)
np.save('/content/content/MyDrive/Sentiment Analysis/test_sentences_new.npy',test_sentences_new)

In [ ]:
# Ví dụ về data, lấy phần tử đầu tiên của tập test

# 0 đại diện cho negative
# 1 đại diện cho 
train_labels[0]

0

In [25]:
# Nén train labels và test label thành numpy array 
# Numpy array là một mảng đa chiều ( ma trận )
train_labels = np.array(train_labels)
test_labels = np.array(test_labels) 


In [26]:
import numpy as np

#Load X,y
train_sentences_new = np.load('/content/content/MyDrive/Sentiment Analysis/train_sentences_new.npy')
test_sentences_new =  np.load('/content/content/MyDrive/Sentiment Analysis/test_sentences_new.npy')

In [27]:

# Định nghĩa hằng số ( constant )

# Số từ trong tập từ vựng
vocab_size = 10000
# Chiều embedding mô phỏng trên không gian 64 chiều ( Chuyển dữ liệu ban đầu thanh một không gian vecto số để huấn luyện mô hình )
# Một từ có 64 thuộc tính để mô tả cho 1 từ
embedding_dim = 64
# Chỉ định độ dài của câu
max_length = 140

In [28]:
# Sử dụng API trong tesorflow để xây dựng từ điển

# Tách từ
from tensorflow.keras.preprocessing.text import Tokenizer 
# Những câu dài ngắn khác nhau thì thu lại thành 1 vevto giống nhau
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [6]:
# Tìm từ nào mà từ đó không có trong từ điển thì gán từ đó cho ký tự: <OOV> và luôn có index bằng 1 
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
# Build từ điển
tokenizer.fit_on_texts(train_sentences_new)

In [7]:
# Index của các từ
tokenizer.word_index
# Index được đánh dựa trên số lần xuất hiện trong tập dữ liệu
# Nếu số lượng xuất hiện bằng nhau thì tính đến từ nào xuất hiện trước thì được đánh index trước

{'<OOV>': 1,
 'i': 2,
 'movie': 3,
 'the': 4,
 'film': 5,
 'one': 6,
 'like': 7,
 'good': 8,
 'this': 9,
 'even': 10,
 'would': 11,
 'time': 12,
 'really': 13,
 'it': 14,
 'story': 15,
 'see': 16,
 'much': 17,
 'get': 18,
 'well': 19,
 'people': 20,
 'also': 21,
 'bad': 22,
 'great': 23,
 'first': 24,
 'dont': 25,
 'made': 26,
 'movies': 27,
 'films': 28,
 'make': 29,
 'could': 30,
 'way': 31,
 'characters': 32,
 'think': 33,
 'watch': 34,
 'but': 35,
 'two': 36,
 'many': 37,
 'seen': 38,
 'character': 39,
 'and': 40,
 'never': 41,
 'little': 42,
 'acting': 43,
 'plot': 44,
 'best': 45,
 'love': 46,
 'its': 47,
 'know': 48,
 'life': 49,
 'show': 50,
 'ever': 51,
 'better': 52,
 'still': 53,
 'a': 54,
 'end': 55,
 'say': 56,
 'in': 57,
 'man': 58,
 'scene': 59,
 'scenes': 60,
 'go': 61,
 'something': 62,
 'back': 63,
 'he': 64,
 'im': 65,
 'doesnt': 66,
 'if': 67,
 'real': 68,
 'watching': 69,
 'years': 70,
 'thing': 71,
 'actors': 72,
 'though': 73,
 'didnt': 74,
 'there': 75,
 'new': 

In [29]:
# Chuyển tập dữ liệu theo index
train_sequences = tokenizer.texts_to_sequences(train_sentences_new)

In [9]:
train_sequences

[[9,
  332,
  305,
  3,
  25,
  1,
  1272,
  3488,
  414,
  1,
  1468,
  23,
  72,
  130,
  233,
  156,
  127,
  405,
  10,
  23,
  43,
  30,
  5460,
  27,
  565,
  665,
  9,
  3,
  314,
  9448,
  89,
  2400,
  326,
  4,
  1139,
  60,
  1,
  7347,
  162,
  2766,
  1,
  2814,
  1,
  1,
  1350,
  4912,
  1,
  1502,
  3488,
  79,
  1139,
  810,
  7802,
  3,
  3984,
  68,
  1166,
  2,
  623,
  27,
  7,
  7241,
  72,
  7,
  1272,
  1,
  8,
  320,
  2,
  30,
  1081,
  769],
 [2,
  536,
  698,
  2280,
  28,
  527,
  585,
  2062,
  98,
  496,
  13,
  1329,
  2349,
  3798,
  1,
  4655,
  87,
  113,
  4152,
  2,
  1431,
  2280,
  5,
  1900,
  4,
  44,
  868,
  1672,
  1223,
  515,
  273,
  98,
  362,
  516,
  1706,
  3893,
  2,
  855,
  2,
  108,
  953,
  90,
  5,
  197,
  2000,
  198,
  362,
  516,
  1,
  115,
  68,
  4440,
  140,
  242,
  2,
  91,
  288,
  5],
 [4749,
  6190,
  1,
  4875,
  3833,
  816,
  1554,
  1924,
  1289,
  2302,
  8672,
  107,
  660,
  266,
  117,
  205,
  35,
  123,
  2

In [10]:
# Chuyển thành các câu có độ dài bằng nhau
# Câu ngắn thì thêm 0, dài thì cắt đi 
# truncating ( xử lí như thế nào khi câu dài hơn )
# padding ( xử lí như thế nào khi câu ngắn hơn )
padded_train_sequences = pad_sequences(train_sequences, maxlen=max_length, truncating='post',padding='post')

In [ ]:
padded_train_sequences

array([[   9,  332,  305, ...,    0,    0,    0],
       [   2,  536,  698, ...,    0,    0,    0],
       [4749, 6190,    1, ...,    0,    0,    0],
       ...,
       [ 844,  189,    2, ...,  325,    1,  572],
       [ 202,    2,  122, ...,    0,    0,    0],
       [  24,    1,  291, ...,   10, 4929,    6]], dtype=int32)

In [11]:
# Số câu để train là 25000
# Độ dài mỗi câu bằng nhau là 140 
padded_train_sequences.shape

(25000, 140)

In [32]:
# Làm tương tự với bộ test
test_sequences = tokenizer.texts_to_sequences(test_sentences_new)
padded_test_sequences = pad_sequences(test_sequences, maxlen= max_length, truncating='post', padding='post')

In [33]:
padded_test_sequences

array([[  75,   28,   29, ...,    0,    0,    0],
       [  54,    1,  643, ...,    1,   37,    1],
       [ 552,    3, 2562, ...,    1, 1769,    1],
       ...,
       [ 826,    1,    3, ...,    0,    0,    0],
       [  47,  514,    2, ...,    0,    0,    0],
       [ 144,   25,   29, ...,    0,    0,    0]], dtype=int32)

In [34]:
padded_test_sequences.shape

(25000, 140)

In [38]:
# Xây dựng model để dự đoán
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
# Với bài toán phân loại thì sử dụng mạng neuron network, trước khi đưa vào sử dụng thì phải duỗi mảng ra
from tensorflow.keras.layers import Flatten,LSTM
from tensorflow.keras.layers import Dense,Dropout

In [45]:
# Create a callback that saves the model's weights
checkpoint_path = "/content/content/MyDrive/Saved_weights/weights.{epoch:04d}.hdf5"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model = Sequential()
# Đưa vào Embedding số từ, số chiều mô phỏng từ đó, định nghĩa đô dài của số câu là 140
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(LSTM(2,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(2,return_sequences=True))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# optimizer: Lựa chọn thuật toán tối ưu
# metrics: Đo lường số câu dự đoán đúng

In [47]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [48]:
model.summary()
# Đầu ra của embedding 140x64, kích cỡ của embedding là chiều dài của vocab_size

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 140, 64)           640000    
                                                                 
 lstm_2 (LSTM)               (None, 140, 2)            536       
                                                                 
 dropout_1 (Dropout)         (None, 140, 2)            0         
                                                                 
 lstm_3 (LSTM)               (None, 140, 2)            40        
                                                                 
 flatten_1 (Flatten)         (None, 280)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                2810      
                                                                 
 dense_3 (Dense)             (None, 1)                

In [ ]:
padded_train_sequences

array([[   9,  332,  305, ...,    0,    0,    0],
       [   2,  536,  698, ...,    0,    0,    0],
       [4749, 6190,    1, ...,    0,    0,    0],
       ...,
       [ 844,  189,    2, ...,  325,    1,  572],
       [ 202,    2,  122, ...,    0,    0,    0],
       [  24,    1,  291, ...,   10, 4929,    6]], dtype=int32)

In [49]:
# Tiến hành train model
model.fit(padded_train_sequences, np.array(train_labels), batch_size =64, epochs=10, validation_data=(padded_test_sequences, np.array(test_labels)),callbacks=[cp_callback])

Epoch 1/10
391/391 [==============================] - ETA: 0s - loss: 0.4068 - acc: 0.8116
Epoch 00001: saving model to /content/content/MyDrive/Saved_weights/weights.0001.hdf5
391/391 [==============================] - 59s 139ms/step - loss: 0.4068 - acc: 0.8116 - val_loss: 0.3400 - val_acc: 0.8552
Epoch 2/10
391/391 [==============================] - ETA: 0s - loss: 0.2352 - acc: 0.9094
Epoch 00002: saving model to /content/content/MyDrive/Saved_weights/weights.0002.hdf5
391/391 [==============================] - 53s 136ms/step - loss: 0.2352 - acc: 0.9094 - val_loss: 0.3991 - val_acc: 0.8414
Epoch 3/10
391/391 [==============================] - ETA: 0s - loss: 0.1787 - acc: 0.9332
Epoch 00003: saving model to /content/content/MyDrive/Saved_weights/weights.0003.hdf5
391/391 [==============================] - 53s 136ms/step - loss: 0.1787 - acc: 0.9332 - val_loss: 0.4126 - val_acc: 0.8424
Epoch 4/10
391/391 [==============================] - ETA: 0s - loss: 0.1346 - acc: 0.9539
Epoch 

In [ ]:
overfit là quá khớp training set, loss train giảm mà loss val tăng, acc train tăng mà acc val giảm, do cái one-hot quá đơn giản nên hz dễ  
overfit ok bitch, gieo vần cl à :v, hoặc có khi cái mô hình phức tạp quá :v, nhưng mà dataset nhiều mà, do cái one-hot :((()))

In [56]:
weight_path = '/content/content/MyDrive/Saved_weights/weights.0003.hdf5'
model.load_weights(weight_path)

test_sen = [" Fucking bitch"]

# Chuyển câu test về index 
test_seq = tokenizer.texts_to_sequences(test_sen) 
# 
padded_test_seq = pad_sequences(test_seq, maxlen=max_length, truncating="post", padding="post")

In [52]:
padded_test_seq

array([[  9, 504,  54,  22,   3,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0]], dtype=int32)

In [57]:
# Đầu ra lớn hơn 0,5 là tích cực và ngược lại
probability = model.predict(padded_test_seq)
if probability[0] >= 0.5: print('Positive')
else: print('Negative')
thôi tạm rứa đi, lưu lại tề lưu lại là ctrl s phăng :v, vcl

Negative
